In [1]:
import re
import os, os.path
import numpy as np
import subprocess
import sys
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import echonet
import wget 
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
model = torchvision.models.video.r2plus1d_18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

In [5]:
DestinationForWeights = "./weights"
StanfordEFWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'
os.path.join(DestinationForWeights, os.path.basename(StanfordEFWeightsURL))

'./weights/r2plus1d_18_32_2_pretrained.pt'

In [6]:
# スタンフォードの重みを取得
DestinationForWeights = "./weights"
StanfordEFWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'

if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(StanfordEFWeightsURL))):
    print("Downloading EF Weights, ", StanfordEFWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(StanfordEFWeightsURL)))
    filename = wget.download(StanfordEFWeightsURL, out = DestinationForWeights)
else:
    print("EF Weights already present")

In [ ]:
print("loading weights from ", os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained.pt"))

if torch.cuda.is_available():
    print("cuda is available, original weights")
#     device = torch.device("cuda")
    device = torch.device(f'cuda:{0}')
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2,3 ]).cuda()
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained.pt"))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)

In [ ]:
print(model)

In [ ]:
# 中間層重みの凍結

# for i, (name, param) in enumerate(model.named_parameters()):
#     if i <= 83:
#         param.requires_grad = False
#     print(i, name)
#     print(param.shape)
#     print(param.requires_grad)
#     print()

In [ ]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [3]:
length = 32
period = 1
n_clips = 2

mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split='train'))

train_dataset = echonet.datasets.Echo(split="train", clips = n_clips, mean=mean, std=std, length=length, period=period)
val_dataset= echonet.datasets.Echo(split="valid", clips = n_clips, mean=mean, std=std, length=length, period=period)
test_dataset = echonet.datasets.Echo(split="test", clips = n_clips, mean=mean, std=std, length=length, period=period)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

100%|██████████| 16/16 [00:01<00:00,  9.33it/s]

1197
462
455


In [ ]:
n_batch_train = 8
n_batch_val = 8

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=n_batch_train, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=n_batch_val, shuffle=True)

In [ ]:
train_loss = []
val_loss = []

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True) 

In [ ]:
# reduce処理ありのauc
for epoch in range(200):
    losses_train = []
    t_true_train = []
    t_pred_train = []
    
    model.train()
    
    for i, (x, y) in enumerate(tqdm(train_dataloader)):

        x_len = x.shape[1]
        t_true_train.extend(y.tolist())
        n_batch = x.shape[0]
        # zero the parameter gradients
        optimizer.zero_grad()
        x = x.reshape(-1, 3, length, 112, 112).to(device)
        y = y.reshape(-1, 1).type(torch.DoubleTensor).to(device)
        # forward + backward + optimize
    
        outputs = model(x)
        
        outputs_reduce = torch.from_numpy(np.zeros((n_batch, 1))).clone().to(device)
        for i in range(n_batch):
            mean = outputs[i*n_clips: i*n_clips+3].mean()
            outputs_reduce[i] = mean

        loss = criterion(outputs_reduce, y)
        loss.backward()
        optimizer.step()

        # print statistics
        losses_train.append(loss.tolist())
        
        outputs_sigmoid = torch.sigmoid(outputs_reduce)
        t_pred_train.extend(outputs_sigmoid.reshape(-1).tolist())

    
    model.eval()
    losses_val = []
    t_true_val = []
    t_pred_val= []
    
    for i, (x, y) in enumerate(tqdm(val_dataloader)):
        x_len = x.shape[1]
        t_true_val.extend(y.tolist())
        n_batch = x.shape[0]
        # zero the parameter gradients
        optimizer.zero_grad()
        x = x.reshape(-1, 3, length, 112, 112).to(device)
        y = y.reshape(-1, 1).type(torch.DoubleTensor).to(device)
        # forward + backward + optimize
    
        outputs = model(x)
        
        outputs_reduce = torch.from_numpy(np.zeros((n_batch, 1))).clone().to(device)
        for i in range(n_batch):
            mean = outputs[i*n_clips: i*n_clips+3].mean()
            outputs_reduce[i] = mean
            
        loss = criterion(outputs_reduce, y)
        # print statistics
        losses_val.append(loss.tolist())
        
        outputs_sigmoid = torch.sigmoid(outputs_reduce)
        t_pred_val.extend(outputs_sigmoid.reshape(-1).tolist())
    
    train_loss.append(np.mean(losses_train))
    val_loss.append(np.mean(losses_val))
    scheduler.step(np.mean(losses_val))
    
    print('EPOCH: {}, Train[{:.3f}, AUC: {:.3f}], Val[{:.3f}, AUC: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        roc_auc_score(t_true_train, t_pred_train),
        np.mean(losses_val),
        roc_auc_score(t_true_val, t_pred_val)
    ))

print('Finished Training')

In [ ]:
# test - with reducing
pred_test_all = []
for i in range(1):
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

    model.eval()
    losses_test = []
    t_true_test = []
    t_pred_test= []

    for i, (x, y) in enumerate(tqdm(test_dataloader)):
            x_len = x.shape[1]
            t_true_test.extend(y.tolist())
            n_batch = x.shape[0]

            # zero the parameter gradients
            optimizer.zero_grad()
            x = x.reshape(-1, 3, length, 112, 112).to(device)
            y = y.reshape(-1, 1).type(torch.DoubleTensor).to(device)
            # forward + backward + optimize

            outputs = model(x)
            outputs_reduce = torch.from_numpy(np.zeros((n_batch, 1))).clone().to(device)
            for i in range(n_batch):
                mean = outputs[i*n_clips: i*n_clips+3].mean()
                outputs_reduce[i] = mean

            loss = criterion(outputs_reduce, y)
            # print statistics
            losses_test.append(loss.tolist())

            outputs_sigmoid = torch.sigmoid(outputs_reduce)
            t_pred_test.extend(outputs_sigmoid.reshape(-1).tolist())

    print('Test{:.3f}, AUC: {:.3f}'.format(
            np.mean(losses_test),
            roc_auc_score(t_true_test, t_pred_test))
        )
    pred_test_all.append(t_pred_test)

In [ ]:
pred_test_all = np.array(pred_test_all).T
pred_test_mean = pred_test_all.mean(axis = 1)
roc_auc_score(t_true_test, pred_test_mean)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(train_loss, linewidth=3, label="train")
plt.plot(val_loss, linewidth=3, label="validation")
plt.title("Learning curve")
plt.ylabel("error")
plt.xlabel("epoch")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(t_true_test, pred_test_mean)
plt.plot(fpr, tpr)
plt.title("TEST AUC : {}".format(str(roc_auc_score(t_true_test, pred_test_mean))))
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
# plt.savefig('data/dst/sklearn_roc_curve.png')

In [ ]:
torch.save(model.state_dict(), "./weights/model1108_sgd_stanford_lr1e5_16_2_3_nb16.pt")